# Imports

In [20]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio # new library
import random #new library
import folium #new library
from simple_colors import * #new library
from pathlib import Path
import json

# Relevant imports for Flights Section
from skyscanner_request import skyscanner_api  # connects to the 'skyscanner_request.py' function file

# Relevant imports for Hotels section
from booking_locations_request import booking_locations_api # connects to the booking_locations_request.py" function file
from booking_hotels_request import booking_hotels_api # connects to the booking_locations_request.py" function file

# Client Data

In [21]:
# Load data on world airports

# Set path to CSV of airports
airports_path = Path('../../resources/datasets/airports_short.csv')
us_cities_path = Path('../../resources/datasets/us_cities_states_counties.csv')

# Read files
csv_df = pd.read_csv(airports_path).set_index('id')
airports_df = csv_df[['name','type','municipality','iata_code','iso_country', 'iso_region']]

csv_df = pd.read_csv(us_cities_path)
us_cities_df = csv_df


In [15]:
# Create new dataframe with airports from the US
us_airports = airports_df.loc[airports_df['iso_country']=="US"]
us_airports.head()

,name,type,municipality,iata_code,iso_country,iso_region
id,,,,,,
18945,Boulder City-Lake Mead International Airport,large_airport,Boulder City,BLD,US,US-NV
18966,Mesquite International Airport,large_airport,Mesquite,MFH,US,US-NV
16091,Albuquerque International Sunport,large_airport,Albuquerque,ABQ,US,US-NM
3362,California Redwood Coast-Humboldt County Inter...,large_airport,Arcata/Eureka,ACV,US,US-CA
3364,Joint Base Andrews,large_airport,Camp Springs,ADW,US,US-MD


In [28]:
# Select two random airports from the us_airports_df

list_airports = us_airports.index.values.tolist()

airports=[]

for i in range(0,2):
    random_airport = random.choice(list_airports)
    airports.append(random_airport)
    print(f"Random airport #{i+1} {us_airports.loc[airports[i]]['iata_code']}")

Random airport #1 FTW
Random airport #2 JFK


In [29]:
# Create new variable with iso_region code for destination (in the case of the US, these would be iso_region codes for the States)
destination_iso_region = airports_df.loc[airports[1]]['iso_region']
destination_iso_region = destination_iso_region[3:]
print(f'Random ISO region: {destination_iso_region}')

# Select a random city located in the iso_region of choice
iso_region_rows = us_cities_df.loc[us_cities_df['State short'] == destination_iso_region]
city_iso_region = random.choice(iso_region_rows['City'].tolist())
print(f'Random city from ISO region: {city_iso_region}')


Random ISO region: NY
Random city from ISO region: East Amherst


In [30]:
# Generate required variables to run APIs

departure_date = "2022-04-12"
return_date = "2022-04-20"
market_country = "US" # country where client is currently present

# Set input variables for skyscanner API request

departure_airport = str(airports_df.loc[airports[0]]['iata_code'])
destination_airport = str(airports_df.loc[airports[1]]['iata_code'])
locale_skyscanner = "en-US"
currency = "USD"

# Set input variables for BOOKING.COM API request
destination_city = city_iso_region
locale_booking = "en-us"
nr_rooms = 1
nr_adults = 1

# SKYSCANNER API Request

In [31]:
# Make an API request from skyscanner

flight_response = skyscanner_api(
    market_country, 
    departure_date, 
    return_date, 
    departure_airport, 
    destination_airport, 
    locale_skyscanner, 
    currency
)

flight_response

{'Quotes': [],
 'Carriers': [],
 'Places': [],
 'Currencies': [{'Code': 'USD',
   'Symbol': '$',
   'ThousandsSeparator': ',',
   'DecimalSeparator': '.',
   'SymbolOnLeft': True,
   'SpaceBetweenAmountAndSymbol': False,
   'RoundingCoefficient': 0,
   'DecimalDigits': 2}]}

In [9]:
# Extract values for itinerary into variables

# Create loop that stores all quotes provided by the API

for quote in range(len(flight_response['Quotes'])):

    if quote == 0:  # If there's only one quote, everything below will be stored in a variable

        flight_price = [flight_response['Quotes'][quote]['MinPrice']]
        airline = [flight_response['Carriers'][quote]['Name']]
        departure_date = flight_response['Quotes'][quote]['OutboundLeg']['DepartureDate']
        departure_airport = flight_response['Places'][0]['Name']
        departure_airport_code = flight_response['Places'][0]['IataCode']
        arrival_airport = flight_response['Places'][1]['Name']
        arrival_airport_code = flight_response['Places'][1]['IataCode']
        if flight_response['Quotes'][quote]['Direct'] == True:
            layover = ["Direct flight"]
        elif flight_response['Quotes'][quote]['Direct'] != True:
            layover = ["Flight with layover(s)"]

    else: # If there is more than one quote, the new flights and airlines will be appended to their respective variables
        new_flight = flight_response['Quotes'][quote]['MinPrice']
        new_airline = flight_response['Carriers'][quote]['Name']
        flight_price.append(new_flight)
        airline.append(new_airline)

        if flight_response['Quotes'][quote]['Direct'] == True:
            new_layover = "Direct flight"
            layover.append(new_layover)
        elif flight_response['Quotes'][quote]['Direct'] != True:
            new_layover = "Flight with layover(s)"
            layover.append(new_layover)


In [10]:
# Create dataframe of quotes

quotes_list = flight_response['Quotes']
flight_quotes_df = pd.json_normalize(quotes_list).set_index('QuoteId') #json_normalize creates extra columns for the nested fields

flight_quotes_df

,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate
QuoteId,,,,,,,
1,104,False,2021-10-25T14:13:00,[851],65368,60987,2021-12-01T00:00:00
2,148,True,2021-10-25T14:13:00,[870],65368,60987,2021-12-01T00:00:00


In [11]:
# Replace the CarrierIds by the Carrier Name on flight_quotes_df
# This code goes through every quote and can handle quotes that have more than one carrier

# Create dataframe of carriers (contains carrier id and name)
carriers_df = pd.DataFrame(flight_response['Carriers']).set_index('CarrierId')


# Create loop that will replace the Carrier Ids with Carrier Names

carriers_row = []
carriers = []

for row in range(len(flight_quotes_df['OutboundLeg.CarrierIds'])):

    # List all carriers for each quote
    quote_carriers = flight_quotes_df['OutboundLeg.CarrierIds'].iloc[row]
    
    # For the row in question: for each carrier, append the Carrier name to a list of carriers in the row
    for carrier in range(len(quote_carriers)):
        carrier_id = quote_carriers[carrier]
        carrier_name = carriers_df.loc[carrier_id]['Name']
        carriers_row.append(carrier_name)
        
    # Store the carriers in each row for all rows
    carriers.append(carriers_row)
    carriers_row = []

# Replace the carrier IDs for carrier names in the flight_quotes_df
flight_quotes_df['OutboundLeg.CarrierIds'] = carriers
flight_quotes_df


,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate
QuoteId,,,,,,,
1,104,False,2021-10-25T14:13:00,[Alaska Airlines],65368,60987,2021-12-01T00:00:00
2,148,True,2021-10-25T14:13:00,[jetBlue],65368,60987,2021-12-01T00:00:00


In [12]:
# Make changes to dataframe

# Rename columns on dataframe
flight_quotes_df.rename(columns={'MinPrice': 'Flight Cost', 'OutboundLeg.CarrierIds': 'Carriers'}, inplace=True)


# Create loop that transforms the list of carriers for each row into a string

rows =[]
for item in range(len(flight_quotes_df['Carriers'])):
    row_values = flight_quotes_df['Carriers'].iloc[item]
    row_as_string = " ".join(map(str,row_values)) #this turns the list [] into a string with all elements
    rows.append(row_as_string)

flight_quotes_df['Carriers'] = rows
flight_quotes_df

,Flight Cost,Direct,QuoteDateTime,Carriers,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate
QuoteId,,,,,,,
1,104,False,2021-10-25T14:13:00,Alaska Airlines,65368,60987,2021-12-01T00:00:00
2,148,True,2021-10-25T14:13:00,jetBlue,65368,60987,2021-12-01T00:00:00


In [13]:
# Plot a comparison of the quotes

flight_quotes_plot = px.bar(flight_quotes_df, 
    x='Carriers', 
    y='Flight Cost',
    title = f"{departure_airport_code} to {arrival_airport_code} Flight Quotes",
    color_discrete_sequence =['orange']*len(flight_quotes_df),
)


flight_quotes_plot.update_layout(
    yaxis_title="Flight Cost (USD)",
    legend_title="Price"
)

flight_quotes_plot

In [ ]:
# Print itinerary quote(s)

if len(flight_response['Quotes']) == 1:  # if there's only one quote, the message below is printed

    print(f"Here's the cheapest flight information we have found:")
    print(f"----------------------------------------------------------")
    print(f"-{layover[quote]}-")
    print(f"{airline} for ${flight_price}, leaving on {departure_date}")
    print(f"Departing from {departure_airport}, {departure_airport_code}")
    print(f"Arriving at {arrival_airport}, {arrival_airport_code}")

elif len(flight_response['Quotes']) > 1: # if there's more than one quote, all options are printed
    
    print(f"Here are some flight options we have found:")
    print(f"----------------------------------------------------------")

    for quote in range(len(flight_response['Quotes'])):
        print(f"Option {quote + 1 }:")
        print(f"-{layover[quote]}-")
        print(f"{airline[quote]} for ${flight_price[quote]}, leaving on {departure_date}")
        print(f"Departing from {departure_airport}, {departure_airport_code}")
        print(f"Arriving at {arrival_airport}, {arrival_airport_code}")
        print(f" ")


In [32]:
# Export the flight quotes to a text file that contains the final results

# Set output file name
output_path = 'flights_output.txt'

# Open the output path as a file object
with open(output_path, 'w') as file:
    if len(flight_response['Quotes']) == 1:  # if there's only one quote, the message below is printed

        file.write(f"Here's the cheapest flight information we have found:\n")
        file.write(f"----------------------------------------------------------\n")
        file.write(f"-{layover[quote]}-\n")
        file.write(f"{airline} for ${flight_price}, leaving on {departure_date}\n")
        file.write(f"Departing from {departure_airport}, {departure_airport_code}\n")
        file.write(f"Arriving at {arrival_airport}, {arrival_airport_code}\n")

    elif len(flight_response['Quotes']) > 1: # if there's more than one quote, all options are printed
    
        file.write(f"Here are some flight options we have found:\n")
        file.write(f"----------------------------------------------------------\n")

        for quote in range(len(flight_response['Quotes'])):
            file.write(f"Option {quote + 1 }:\n")
            file.write(f"-{layover[quote]}-\n")
            file.write(f"{airline[quote]} for ${flight_price[quote]}, leaving on {departure_date}\n")
            file.write(f"Departing from {departure_airport}, {departure_airport_code}\n")
            file.write(f"Arriving at {arrival_airport}, {arrival_airport_code}\n")
            file.write(f" \n")


# BOOKING.COM API Requests

In [33]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")

In [34]:
# Make API request from Booking.com for number of hotels in the city by location (districts, landmarks, areas, etc.)
location_endpoint_response = booking_locations_api(destination_city, locale_booking)


NameError: name 'city' is not defined

In [6]:
# Make changes to the dataframe
locations_df = pd.DataFrame.from_dict(location_endpoint_response).set_index('dest_id')
locations_df.drop(columns = ['country', 'city_ufi', 'landmark_type', 'region','rtl', 'timezone', 'lc', 'cc1', 'image_url', 'hotels', 'type'], inplace =True)
locations_df

,longitude,label,nr_hotels,latitude,city_name,dest_type,name
dest_id,,,,,,,
20088325,-73.981895,"New York, New York State, United States",1382,40.768074,New York,city,New York
2306,-73.984912,"Central New York City, New York, New York Stat...",258,40.759591,New York,district,Central New York City
929,-73.970891,"Manhattan, New York, New York State, United St...",647,40.776115,New York,district,Manhattan
1397916,-73.988439,"Riu Plaza New York Times Square, New York, New...",1,40.760248,New York,hotel,Riu Plaza New York Times Square
973,-73.965827,"New York Central Park, New York, New York Stat...",54,40.782019,New York,district,New York Central Park


In [7]:
# Set the mapbox access token
px.set_mapbox_access_token(map_box_api)

# Create a scatter mapbox to analyze neighborhood info
locations_map_plot = px.scatter_mapbox(
    locations_df,
    lat="latitude",
    lon="longitude",
    width=1000, 
    height=1000,
    opacity=0.9,
    color = locations_df['dest_type'],
    size = locations_df['nr_hotels'],
    hover_name = locations_df['name'],
    hover_data = {'name':True, 'dest_type': True, 'nr_hotels':True, 'longitude': False, 'latitude': False, 'city_name': False},
    zoom = 11
)

locations_map_plot.show()

In [8]:
# Make API request from Booking.com for hotels in a specific dest_id (districts, landmarks, areas, etc.)

# Select a random dest_id from the locations_df
list_dest_ids = locations_df.index.values.tolist()
random_dest_id = random.choice(list_dest_ids)
dest_type = locations_df.loc[random_dest_id]['dest_type']

# Run API request
hotels_endpoint_response = booking_hotels_api(dest_type, departure_date, return_date, nr_rooms, locale_booking, random_dest_id, nr_adults)



In [130]:
# Generate dataframe with hotel results

hotels_df = pd.DataFrame.from_dict(hotels_endpoint_response["result"])

# Filter out desired columns from dataframe
desired_cols = ["hotel_id","hotel_name","min_total_price","distance_to_cc", "district","address","zip","review_score_word","review_score","checkout","latitude","longitude","urgency_message","url"]
hotels_df = hotels_df[hotels_df.columns.intersection(desired_cols)]

# Organize dataframe columns, sort by price(lowest to highest) and set index to hotel_id
hotels_df = hotels_df.reindex(columns=desired_cols).set_index('hotel_id').sort_values('min_total_price')
#hotels_df['min_total_price'] = hotels_df['min_total_price'].map('${:,.2f}'.format)
hotels_df.head(5)

,hotel_name,min_total_price,distance_to_cc,district,address,zip,review_score_word,review_score,checkout,latitude,longitude,urgency_message,url
hotel_id,,,,,,,,,,,,,
7366978,Nap York Youth Hostel,1070.912500,0.30,Manhattan,940 8th Avenue 4th floor,NY 10019,Very Good,8.0,"{'from': '08:00', 'until': '10:00'}",40.765665,-73.983232,Only 4 left at this price on Booking.com,https://www.booking.com/hotel/us/nap-york-new-...
1106729,West Side YMCA,1371.200000,0.35,Upper West Side,5 West 63rd Street,NY 10023,Good,7.3,"{'from': '', 'until': '11:00'}",40.770845,-73.980612,NaN,https://www.booking.com/hotel/us/west-side-ymc...
2039182,Park West Hotel,3083.104750,3.90,Upper West Side,465 Central Park West,NY 10025,Pleasant,6.5,"{'until': '12:30', 'from': '12:00'}",40.798527,-73.960044,Only 3 left at this price on Booking.com,https://www.booking.com/hotel/us/astor-on-cent...
182507,La Quinta by Wyndham New York City Central Park,3699.943375,1.00,Upper West Side,31 West 71st Street,NY 10023,Good,7.3,"{'until': '12:00', 'from': ''}",40.776145,-73.977684,NaN,https://www.booking.com/hotel/us/la-quinta-new...
46601,Sonder l Chambers,4803.320250,0.80,Manhattan,15 West 56th Street,NY 10019,Excellent,8.8,"{'from': '', 'until': '11:00'}",40.762866,-73.975599,Only 1 left at this price on Booking.com,https://www.booking.com/hotel/us/chambers.html


In [10]:
hotel_pricing_plot = px.scatter(
    hotels_df,
    x= hotels_df['min_total_price'],
    y = hotels_df['review_score'],
    color = hotels_df['district'],
    size = hotels_df['min_total_price'], 
    hover_name=hotels_df["hotel_name"],
    labels=dict(review_score="Review Scores", district="District", min_total_price = "Price in USD for entire stay"),
    title="Hotel Pricing and Reviews by District" 
)

hotel_pricing_plot.update_traces(hovertemplate ='<b>Price</b>: $%{x:.2f} <br> <b>Score</b>: %{y}')

hotel_pricing_plot

In [11]:
# Use folium to create Map of hotels in dest_id of choice (district, landmark, city, etc.)

# Creates map around a set of coordinates from the dataframe
start_coords = [hotels_df['latitude'].iloc[0], hotels_df['longitude'].iloc[0]]
m = folium.Map(location=start_coords, zoom_start=12)


# Create loop that adds all hotel coordinates to the map
coords = [hotels_df['latitude'].to_list(), hotels_df['longitude'].to_list()]

for i in range(len(coords[0])):
    folium.Marker(location= [coords[0][i], coords[1][i]],
    tooltip= hotels_df['hotel_name'].iloc[i],
    icon=folium.Icon(icon='bed', prefix='fa'),
    popup =f"{hotels_df['review_score'].iloc[i]}/10"
    ).add_to(m) 
 
hotels_map = m
hotels_map

# icon examples for hotels: "bed" "h-square" these two need the prefix 'fa' for font awesome website (recommended by folium documentation)

In [170]:
# Print hotel quotes

print(f"Here are some hotel options for you:")
print(f"----------------------------------------------------------")

for quote in range(len(hotels_df)):
    print(green(f"{hotels_df['hotel_name'].iloc[quote]}"), f"({hotels_df['review_score'].iloc[quote]}/10.0)")
    print(f"{hotels_df['address'].iloc[quote]} {hotels_df['zip'].iloc[quote]}")
    print(f"Price: ${hotels_df['min_total_price'].iloc[quote]:.2f}")
    print(blue(f"{hotels_df['url'].iloc[quote]}"))

    #this loop checks if there is an urgency_message for the hotel, if present, it prints the message
    if type(hotels_df['urgency_message'].iloc[quote])==str:
        print(f"{hotels_df['urgency_message'].iloc[quote]}")
    elif type(hotels_df['urgency_message'].iloc[quote])==float:
        pass

    print(f" ")

Here are some hotel options for you:
----------------------------------------------------------
Nap York Youth Hostel (8.0/10.0)
940 8th Avenue 4th floor NY 10019
Price: $1070.91
https://www.booking.com/hotel/us/nap-york-new-york1.html
Only 4 left at this price on Booking.com
 
West Side YMCA (7.3/10.0)
5 West 63rd Street NY 10023
Price: $1371.20
https://www.booking.com/hotel/us/west-side-ymca.html
 
Park West Hotel (6.5/10.0)
465 Central Park West NY 10025
Price: $3083.10
https://www.booking.com/hotel/us/astor-on-central-park.html
Only 3 left at this price on Booking.com
 
La Quinta by Wyndham New York City Central Park (7.3/10.0)
31 West 71st Street NY 10023
Price: $3699.94
https://www.booking.com/hotel/us/la-quinta-new-york-city-central-park.html
 
Sonder l Chambers (8.8/10.0)
15 West 56th Street NY 10019
Price: $4803.32
https://www.booking.com/hotel/us/chambers.html
Only 1 left at this price on Booking.com
 
AKA Central Park (8.6/10.0)
42 West 58th Street NY 10019
Price: $4816.18
h

In [185]:
# Export the analysis to a text file that contains the final results

# Set output file name
output_path = 'hotels_output.txt'

# Open the output path as a file object
with open(output_path, 'w') as file:
    file.write(f"Here are some hotel options for you:\n")
    file.write(f"----------------------------------------------------------\n")

    for quote in range(len(hotels_df)):
        file.write(f"{hotels_df['hotel_name'].iloc[quote]}\n") 
        file.write(f"{hotels_df['review_score'].iloc[quote]}/10.0)\n")
        file.write(f"{hotels_df['address'].iloc[quote]} {hotels_df['zip'].iloc[quote]}\n")
        file.write(f"Price: ${hotels_df['min_total_price'].iloc[quote]:.2f}\n")
        file.write(f"{hotels_df['url'].iloc[quote]}\n")

        #this loop checks if there is an urgency_message for the hotel, if present, it prints the message
        if type(hotels_df['urgency_message'].iloc[quote])==str:
            file.write(f"{hotels_df['urgency_message'].iloc[quote]}\n")
        elif type(hotels_df['urgency_message'].iloc[quote])==float:
            pass

        file.write(f" \n")